
# Fit a ridge model with motion-energy features

In this example, we model the fMRI responses with motion-energy features
extracted from the movie stimulus. The model is a regularized linear regression
model.

This tutorial reproduces part of the analysis described in {cite:t}`nishimoto2011`. See the original publication for more details about the experiment, the
motion-energy features, along with more results and more discussions.

*Motion-energy features:* Motion-energy features result from filtering a video
stimulus with spatio-temporal Gabor filters. A pyramid of filters is used to
compute the motion-energy features at multiple spatial and temporal scales.
Motion-energy features were introduced in {cite:t}`nishimoto2011`. The downloaded 
dataset contains the pre-computed motion-energy features for the movie stimulus used 
in the experiment. You can see how to extract these motion-energy features in the
[Extract motion-energy features](07_extract_motion_energy.ipynb) tutorial.

*Summary:* As in the previous example, we first concatenate the features with
multiple delays, to account for the slow hemodynamic response. A linear
regression model then weights each delayed feature with a different weight, to
build a predictive model of BOLD activity. Again, the linear regression is
regularized to improve robustness to correlated features and to improve
generalization. The optimal regularization hyperparameter is selected
independently on each voxel over a grid-search with cross-validation. Finally,
the model generalization performance is evaluated on a held-out test set,
comparing the model predictions with the ground-truth fMRI responses.

:::{note}
It should take less than 5 minutes to run the model fitting in this tutorial on a GPU. If you are using a CPU, it may take longer.
:::

## Path of the data directory



In [ ]:
from voxelwise_tutorials.io import get_data_home
directory = get_data_home(dataset="shortclips")
print(directory)

In [ ]:
# modify to use another subject
subject = "S01"

## Load the data

We first load the fMRI responses.



In [ ]:
import os
import numpy as np
from voxelwise_tutorials.io import load_hdf5_array

file_name = os.path.join(directory, "responses", f"{subject}_responses.hdf")
Y_train = load_hdf5_array(file_name, key="Y_train")
Y_test = load_hdf5_array(file_name, key="Y_test")

print("(n_samples_train, n_voxels) =", Y_train.shape)
print("(n_repeats, n_samples_test, n_voxels) =", Y_test.shape)

We average the test repeats, to remove the non-repeatable part of fMRI
responses.



In [ ]:
Y_test = Y_test.mean(0)

print("(n_samples_test, n_voxels) =", Y_test.shape)

We fill potential NaN (not-a-number) values with zeros.



In [ ]:
Y_train = np.nan_to_num(Y_train)
Y_test = np.nan_to_num(Y_test)

Then we load the precomputed "motion-energy" features.



In [ ]:
feature_space = "motion_energy"
file_name = os.path.join(directory, "features", f"{feature_space}.hdf")
X_train = load_hdf5_array(file_name, key="X_train")
X_test = load_hdf5_array(file_name, key="X_test")

print("(n_samples_train, n_features) =", X_train.shape)
print("(n_samples_test, n_features) =", X_test.shape)

## Define the cross-validation scheme

We define the same leave-one-run-out cross-validation split as in the
previous example.



In [ ]:
from sklearn.model_selection import check_cv
from voxelwise_tutorials.utils import generate_leave_one_run_out

# indice of first sample of each run
run_onsets = load_hdf5_array(file_name, key="run_onsets")
print(run_onsets)

We define a cross-validation splitter, compatible with ``scikit-learn`` API.



In [ ]:
n_samples_train = X_train.shape[0]
cv = generate_leave_one_run_out(n_samples_train, run_onsets)
cv = check_cv(cv)  # copy the cross-validation splitter into a reusable list

## Define the model

We define the same model as in the previous example. See the previous
example for more details about the model definition.



In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from voxelwise_tutorials.delayer import Delayer
from himalaya.kernel_ridge import KernelRidgeCV
from himalaya.backend import set_backend
backend = set_backend("torch_cuda", on_error="warn")

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

alphas = np.logspace(1, 20, 20)

pipeline = make_pipeline(
    StandardScaler(with_mean=True, with_std=False),
    Delayer(delays=[1, 2, 3, 4]),
    KernelRidgeCV(
        alphas=alphas, cv=cv,
        solver_params=dict(n_targets_batch=500, n_alphas_batch=5,
                           n_targets_batch_refit=100)),
)

In [ ]:
from sklearn import set_config
set_config(display='diagram')  # requires scikit-learn 0.23
pipeline

## Fit the model

We fit on the train set, and score on the test set.



In [ ]:
pipeline.fit(X_train, Y_train)

scores_motion_energy = pipeline.score(X_test, Y_test)
scores_motion_energy = backend.to_numpy(scores_motion_energy)

print("(n_voxels,) =", scores_motion_energy.shape)

## Plot the model performances
The performances are computed using the $R^2$ scores.



In [ ]:
import matplotlib.pyplot as plt
from voxelwise_tutorials.viz import plot_flatmap_from_mapper

mapper_file = os.path.join(directory, "mappers", f"{subject}_mappers.hdf")
ax = plot_flatmap_from_mapper(scores_motion_energy, mapper_file, vmin=0,
                              vmax=0.5)
plt.show()

The motion-energy features lead to large generalization scores in the
early visual cortex (V1, V2, V3, ...). For more discussions about these
results, we refer the reader to the original publication {cite}`nishimoto2011`.

## Compare with the wordnet model

Interestingly, the motion-energy model performs well in different brain
regions than the semantic "wordnet" model fitted in the previous example. To
compare the two models, we first need to fit again the wordnet model.

In [ ]:
feature_space = "wordnet"
file_name = os.path.join(directory, "features", f"{feature_space}.hdf")
X_train = load_hdf5_array(file_name, key="X_train")
X_test = load_hdf5_array(file_name, key="X_test")

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

We can create an unfitted copy of the pipeline with the ``clone`` function,
or simply call fit again if we do not need to reuse the previous model.



In [ ]:
if False:
    from sklearn.base import clone
    pipeline_wordnet = clone(pipeline)
    pipeline_wordnet

In [ ]:
pipeline.fit(X_train, Y_train)
scores_wordnet = pipeline.score(X_test, Y_test)
scores_wordnet = backend.to_numpy(scores_wordnet)

ax = plot_flatmap_from_mapper(scores_wordnet, mapper_file, vmin=0,
                              vmax=0.5)
plt.show()

We can also plot the comparison of model prediction accuracies with a 2D
histogram. All ~70k voxels are represented in this histogram, where the
diagonal corresponds to identical prediction accuracy for both models. A
distribution deviating from the diagonal means that one model has better
predictive performance than the other.



In [ ]:
from voxelwise_tutorials.viz import plot_hist2d

ax = plot_hist2d(scores_wordnet, scores_motion_energy)
ax.set(title='Generalization R2 scores', xlabel='semantic wordnet model',
       ylabel='motion energy model')
plt.show()

Interestingly, the well predicted voxels are different in the two models.
To further describe these differences, we can plot both performances on the
same flatmap, using a 2D colormap.



In [ ]:
from voxelwise_tutorials.viz import plot_2d_flatmap_from_mapper

mapper_file = os.path.join(directory, "mappers", f"{subject}_mappers.hdf")
ax = plot_2d_flatmap_from_mapper(scores_wordnet, scores_motion_energy,
                                 mapper_file, vmin=0, vmax=0.25, vmin2=0,
                                 vmax2=0.5, label_1="wordnet",
                                 label_2="motion energy")
plt.show()

The blue regions are well predicted by the motion-energy features, the orange
regions are well predicted by the wordnet features, and the white regions are
well predicted by both feature spaces.

A large part of the visual semantic areas are not only well predicted by the
wordnet features, but also by the motion-energy features, as indicated by the
white color. Since these two features spaces encode quite different
information, two interpretations are possible. In the first interpretation,
the two feature spaces encode complementary information, and could be used
jointly to further increase the generalization performance. In the second
interpretation, both feature spaces encode the same information, because of
spurious stimulus correlations. For example, imagine that the visual stimulus
contained faces that appeared consistetly in the same portion of the visual
field. In this case, position in the visual field would be perfectly
correlated with the "face" semantic category. Thus, motion-energy features
could predict responses in face-responsive areas without encoding any
semantic information.

To better disentangle the two feature spaces, we developed a joint model
called `banded ridge regression` {cite}`nunez2019,dupre2022`, which fits multiple feature spaces
simultaneously with optimal regularization for each feature space. This model
is described in the next example.



## References

```{bibliography}
:filter: docname in docnames
```